In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [369]:
file_number = 'c5'

### Read Document

In [370]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [371]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [372]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [373]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)

### Get Coref for threshold match by NP

In [374]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)

### Get Coref for Word Embedding Similarity by NP

In [375]:
coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [376]:
coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging coref_dict_match_NP with coref_dict_all_sorted_top3

In [377]:
def mergeDicts(string_matching_dict, word_embedding_dict):
    
    d1 = string_matching_dict.copy()
    d2 = word_embedding_dict.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return d1

In [378]:
coref_final = mergeDicts(coref_dict_match_NP, coref_dict_all_sorted)

In [379]:
#coref_final

### Passing the reference dict to Hobbs

In [380]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [381]:
# coref_final_with_pro

### Adding those Cluster Heads which were not predicted

In [390]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [391]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

### Removing the Determiners from the start

In [392]:
def removeDetfromStart(coref_final_with_pro):
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            if(len(coref_final_with_pro[cluster][i][2].split(' '))>1):
                ref_word_list = coref_final_with_pro[cluster][i][2].strip().split(' ')
                ref_word_list = list(filter(bool, ref_word_list) )
                check = pos_tag(ref_word_list)
                if len(check) >1:
                    if 'DT' in check[0][1]:
                        sentence = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                        coref_final_with_pro[cluster][i][2] = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                        
    return coref_final_with_pro

In [393]:
coref_final_with_pro = removeDetfromStart(coref_final_with_pro)

### Making no predictions for Pronouns (will be included in final version - Reverse Hobb's/ Trained Word Embedding model)

In [395]:
def removePredsforPronouns(coref_final_with_pro):
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            curr_cluster_name = coref_final_with_pro[cluster][i][0].split(' ')
            if(len(curr_cluster_name)==1 and curr_cluster_name[0]!=''):
                if('PRP' in pos_tag(curr_cluster_name)[0]):
                    coref_final_with_pro[cluster] = []
                    break
    return coref_final_with_pro

In [396]:
coref_final_with_pro = removePredsforPronouns(coref_final_with_pro)

### Print Output

In [385]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [386]:
%%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

In [387]:
with open('{}.response'.format(file_number),'w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans